**Cross-Encoder Re-Ranking**

Install sentence-transformers python library  and import 'CrossEncoder' from this library.

In [1]:
# place your code here
import sys
!{sys.executable} -m pip install sentence-transformers


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# place your code here
from sentence_transformers import CrossEncoder

C:\Users\renas.alp\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From typing  import List, Tuple

In [3]:
# place your code here
from typing import List, Tuple

Load the pre-trained cross-encoder model 'cross-encoder/ms-marco-MiniLM-L-6-v2' and save the model object to the 'model' variable. To load, use CrossEncoder() with two arguments: the first is the model name as string and the second argument is 'max_length=512'

In [4]:
# place your code here
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

c:\Users\renas.alp\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\renas.alp\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 105/105 [00:00<00:00, 262.34it/s, M

Use function:

In [5]:
def rerank_documents(
    query: str,  documents: List[str],  top_k: int = 10
) -> List[Tuple[int, str, float]]:
    """
    Re-rank documents based on relevance to query.
    Args:
        query: The search query
        documents: List of document texts from initial retrieval
        top_k: Number of top results to return
    Returns:
        List of (original_index, document, score) tuples, sorted by relevance
    """
    # Create query-document pairs for the cross-encoder
    # Each pair will be scored independently
    pairs = [[query, doc] for doc in documents]

    # Get relevance scores for all pairs
    # The model outputs a single score per pair
    scores = model.predict(pairs)

    # Combine with original indices and sort by score descending
    scored_docs = [
        (idx, doc, float(score))
        for idx, (doc, score) in enumerate(zip(documents, scores))
    ]
    scored_docs.sort(key=lambda x: x[2], reverse=True)

    return scored_docs[:top_k]

Simulate retrieval results: use the following data.

In [6]:
retrieved_docs = [
        "Kubernetes pod monitoring requires metrics collection from the kubelet.",
        "Docker containers can be monitored using cAdvisor metrics.",
        "To check pod status, use kubectl get pods command.",
        "Prometheus is commonly used for Kubernetes monitoring.",
        "Pod resource limits should be set in the deployment spec.",
        "OneUptime provides real-time Kubernetes monitoring dashboards.",
        "Container orchestration platforms need observability solutions.",
        "The kubectl logs command shows container output.",
    ]
query = "How do I monitor Kubernetes pods?"

Select a value for the k parameter and apply the "rerank_documents()" function to the data.
Save the returned object to the results variabl.

In [8]:
k = 7
results = rerank_documents(query, retrieved_docs, top_k=k)

Using the 'for idx, doc, score in results'    loop, print the values ​​of 'idx', 'doc' and 'score'

In [15]:
print("Re-ranked results:")
for idx, doc, score in results:
    print("Index:", idx)
    print("Document:", doc)
    print("Score:", score)
    print("-"*70)

Re-ranked results:
Index: 0
Document: Kubernetes pod monitoring requires metrics collection from the kubelet.
Score: 6.1957926750183105
----------------------------------------------------------------------
Index: 5
Document: OneUptime provides real-time Kubernetes monitoring dashboards.
Score: 2.8766822814941406
----------------------------------------------------------------------
Index: 3
Document: Prometheus is commonly used for Kubernetes monitoring.
Score: 2.8236849308013916
----------------------------------------------------------------------
Index: 2
Document: To check pod status, use kubectl get pods command.
Score: 0.14170622825622559
----------------------------------------------------------------------
Index: 7
Document: The kubectl logs command shows container output.
Score: -8.542131423950195
----------------------------------------------------------------------
Index: 1
Document: Docker containers can be monitored using cAdvisor metrics.
Score: -10.013574600219727
-----